# Theano More Examples

- Theano의 근본적인 목적과 동적을 보다 체계적으로 알아보자.
- 좀더 활용분야를 넓어보자.

## Logistic Function

![logistic](http://deeplearning.net/software/theano/_images/math/ad5550567b9b0eae6cf861ba9704d98d36dd32a6.png)

![logistic](http://deeplearning.net/software/theano/_images/logistic.png)

- x축은 x값,  Y축은 s(x)값
- matrix의 각각 개별 요소별로 function이 적용된다는 의미로 function이 elementwise 하게 계산됨.

In [2]:
import theano
import theano.tensor as T
x = T.dmatrix('x')
s = 1 / (1 + T.exp(-x))

In [3]:
logistic = theano.function([x], s)

In [4]:
logistic([[0, 1], [-1, -2]])

array([[ 0.5       ,  0.73105858],
       [ 0.26894142,  0.11920292]])

![tanh](http://deeplearning.net/software/theano/_images/math/170dc7d9a165d8cd72fe9c35973e10b5d103fce9.png)

In [7]:
s2 = (1 + T.tanh(x / 2)) / 2
logistic2 = theano.function([x], s2)
logistic2([[0, 1], [-1, -2]])

array([[ 0.5       ,  0.73105858],
       [ 0.26894142,  0.11920292]])

## 한번에 여러가지 계산하기
- Theano는 여러개의 결과물 가지는 function을 지원함.
- 개별요소별로 a와 b의 행렬간의 차이, 차의 절대값, 차의 제곱을 한번에 계산

In [8]:
a, b = T.dmatrices('a', 'b')
diff = a - b
abs_diff = abs(diff)
diff_squared = diff**2
f = theano.function([a, b], [diff, abs_diff, diff_squared])

In [10]:
f([[1, 1], [1, 1]], [[0, 1], [2, 3]] )

[array([[ 1.,  0.],
        [-1., -2.]]), array([[ 1.,  0.],
        [ 1.,  2.]]), array([[ 1.,  0.],
        [ 1.,  4.]])]

In [ ]:
[array([[ 1.,  0.], [-1., -2.]]), 
 array([[ 1.,  0.], [ 1.,  2.]]), 
 array([[ 1.,  0.], [ 1.,  4.]])]

## function의 인자에 기본값 설정
- 두개 숫자값을 더하는 함수를 만들고, 만약 하나의 숫자만 넣으면 다른 숫자값은 1 이라고 가정하자.
- Param 클래스를 이용해서 y 변수에 기본값(default)을 1로 설정함.

In [11]:
from theano import Param
from theano import function
x, y = T.dscalars('x', 'y')
z = x + y
f = function([x, Param(y, default=1)], z)

In [12]:
f(33)

array(34.0)

In [13]:
f(33, 2)

array(35.0)

- 여러개의 입력변수에 여러 개의 default값을 갖을 수 있음.
- Python과 같이 위치나 이름(name)으로 입력값이 어떤 변수인지 지정할 수 있음.

In [14]:
x, y, w = T.dscalars('x', 'y', 'w')
z = (x + y) * w
f = function([x, Param(y, default=1), Param(w, default=2, name='w_by_name')], z)

In [15]:
f(33)

array(68.0)

In [17]:
f(33, 2)

array(70.0)

In [18]:
f(33, 0, 1)

array(33.0)

In [19]:
f(33, w_by_name=1)

array(34.0)

In [20]:
f(33, w_by_name=1, y=0)

array(33.0)

## Using Shared Variables
- 내부적인 상태를 저장하는 function를 만드는 것이 가능함.
- accumulator라는 function을 만들고 처음 시작할때 상태값을 0으로 설정하고 function이 호출할때마다 상태값을 증가시켜보자.

In [49]:
from theano import shared
state = shared(0)
inc = T.iscalar('inc')
accumulator = function([inc], state, updates=[(state, state+inc)])

- 2가지 새로운 개념이 포함됨


- shared() function은 shared variables(공유변수)을 생성함.
- 공유변수는 여러 function들 사이에서 값을 공유할 수 있게 하며, symbolic이 없을 수 있고, 있을 수 있음.
- 공유변수는 dmatrices(...) 와 같이 symbolic 로 표현할 수 있지만, 내부적으로 symbolic variable(기호가 있는 변수)을 생성하고 이를 사용함.
- 공유변수는 여러 function들에서 공유될 수 있기 때문에 이렇게 불리어짐.
- .get_value() method로 값을 읽고, .set_value() method로 값을 수정함.<br>


- function의 updates 인자가 새롭게 등장
- updates는 (shared변수, 표현식) 쌍을 이루는 형식들의 리스트를 갖음.
- python의 dictionary 와 비슷하게 키가 shared변수이고 값이 표현식 이라고 할 수 있음.
- updates의 의미는 funtion이 실행될때마다, shared변수의 값이  표현식의 결과값으로 대체됨.

In [50]:
state.get_value()

array(0)

In [51]:
accumulator(1)

array(0)

In [52]:
state.get_value()

array(1)

In [53]:
accumulator(300)

array(1)

In [54]:
state.get_value()

array(301)

In [55]:
# .set_value() method을 사용해서 상태값을 -1 초기화 
state.set_value(-1)

In [56]:
accumulator(3)

array(-1)

In [57]:
state.get_value()

array(2)

- 같은 공유변수를 사용하여 다른 function을 정의하고 그 function으로 공유변수값을 변경해보자.

In [58]:
decrementor = function([inc], state, updates=[(state, state-inc)])
decrementor(2)

array(2)

In [59]:
state.get_value()

array(0)

In [60]:
accumulator(3)

array(0)

In [61]:
state.get_value()

array(3)

- updates 메카니즘이 존재하는 이유는 무엇인가?
- 새로운 표현식으로 비슷한 결과를 얻기 위함, numpy와 비슷. ???
- 다른 중요한 이유는 효율성 때문 
- Theano에서는 공유변수를 사용하여 GPU를 활용함.


- 공유변수를 사용하는 새로운 표현식을 만들고, 공유변수를 변경하지 않도록 하자.
- 이 경우에는 givens 인자를 사용
- givens 는 특정한 기능을 목적으로 graph에서 특정노드값을 대체함. ????

In [62]:
fn_of_state = state * 2 + inc
# The type of foo must match the shared variable we are replacing  with the ``givens``
foo = T.scalar(dtype=state.dtype)
skip_shared = function([inc, foo], fn_of_state, givens=[(state, foo)])

In [63]:
skip_shared(1, 3)  # we're using 3 for the state, not state.value

array(7)

In [64]:
state.get_value()  # old state still there, but we didn't use it

array(3)

- In practice, a good way of thinking about the givens is as a mechanism that allows you to replace any part of your formula with a different expression that evaluates to a tensor of same shape and dtype.

## Using Random Numbers

In [ ]:
case class SimpleRNG( seed: Long) extends RNG {
    def nextInt : ( Int, RNG ) = {
        val newSeed = ( seed * 0x5DEECE66DL + 0xBL )  & 0xFFFFFFFFFFFFL
        val nextRNG = SimpleRNG(newSeed)
        val n = ( newSeed >>> 16 ).toInt
        ( n, nextRNG)
    }
}

- 프로그램상의 random값은 위의 함수에서 보는것 같이 간단한 수식에 의해서 생성되고,  seed값이 동일하면 항상 같은 값을 생성함.
- 그러므로, random테이블이라고 생각하면 편리함. 큰 배열에 어떤 값들이 무작위로 들어있고, seed값은 그 배열의 위치값이고, 그 위치값에는 항상 동일한 값임.

### Brief Example

In [107]:
from theano.tensor.shared_randomstreams import RandomStreams
from theano import function
srng = RandomStreams(seed=234)
rv_u = srng.uniform((2,2))
rv_n = srng.normal((2,2))
f = function([], rv_u)
g = function([], rv_n, no_default_updates=True)    #Not updating rv_n.rng
nearly_zeros = function([], rv_u + rv_u - 2 * rv_u)

In [108]:
f_val0 = f()

In [109]:
f_val1 = f()  #different numbers from f_val0

In [110]:
f_val0 

array([[ 0.12672381,  0.97091597],
       [ 0.13989098,  0.88754825]])

In [111]:
f_val1 

array([[ 0.31971415,  0.47584377],
       [ 0.24129163,  0.42046081]])

In [112]:
g_val0 = g()  # different numbers from f_val0 and f_val1
g_val1 = g()  # same numbers as g_val0!

In [113]:
g_val0

array([[ 0.37328447, -0.65746672],
       [-0.36302373, -0.97484625]])

In [114]:
g_val1

array([[ 0.37328447, -0.65746672],
       [-0.36302373, -0.97484625]])

### Seeding Streams

In [115]:
rng_val = rv_u.rng.get_value(borrow=True)   # Get the rng for rv_u
rng_val

In [116]:
rng_val.seed(89234)                         # seeds the generator

In [117]:
rv_u.rng.set_value(rng_val, borrow=True)    # Assign back seeded rng

In [74]:
srng.seed(902340)  # seeds rv_u and rv_n with different seeds each

### Sharing Streams Between Functions

In [123]:
state_after_v0 = rv_u.rng.get_value().get_state()
state_after_v0

('MT19937', array([2899575347,  533481454, 2091657755,  618751085, 3298364243,
        3150007752, 1093551589, 1100629552, 1826491916,  637690239,
        1943378584, 2360375819, 1050865619, 2191829958, 2648960233,
        3030813155,  439596985, 1444403372, 2047496234, 2151205240,
        4133379565,  995674357, 3757891888,  522631282, 3722383727,
        3449033878, 2134389890, 2614243788, 1474112254, 1195862375,
         579178790, 2816899363,  368085582, 4122430563, 3437551073,
         137033961,  211635088,  392171372,  720348031, 1984232378,
        2853641342, 1067425262, 2050244902, 2372420588, 2537430678,
        3232956770,  721063989, 4277580032, 3649021159,  940963481,
        1041111370, 1774257140,  434367250, 2535732461, 3364046525,
        2437785266,  953619758, 3969027395,   30570048, 1480842503,
        4026264310,  742584634, 2025288578, 4110481674, 2377551597,
        1083303260, 2571029229, 2228469276, 2201249257, 1889428461,
         985144475,  895319134,  2737

In [124]:
nearly_zeros()

array([[ 0.,  0.],
       [ 0.,  0.]])

In [125]:
v1 = f()
v1

array([[ 0.65295707,  0.81246009],
       [ 0.43266108,  0.84805177]])

In [126]:
rng = rv_u.rng.get_value(borrow=True)
rng.set_state(state_after_v0)
rv_u.rng.set_value(rng, borrow=True)

In [127]:
v2 = f()             # v2 != v1
v2

array([[ 0.52240108,  0.47792506],
       [ 0.40217335,  0.44828264]])

In [128]:
v3 = f()             # v3 == v1
v3

array([[ 0.65295707,  0.81246009],
       [ 0.43266108,  0.84805177]])

### Copying Random State Between Theano Graphs

In [129]:
from __future__ import print_function
import theano
import numpy
import theano.tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams
from theano.tensor.shared_randomstreams import RandomStreams

In [130]:
class Graph():
    def __init__(self, seed=123):
        self.rng = RandomStreams(seed)
        self.y = self.rng.uniform(size=(1,))

In [131]:
g1 = Graph(seed=123)
f1 = theano.function([], g1.y)

In [132]:
g2 = Graph(seed=987)
f2 = theano.function([], g2.y)

In [133]:
f1()

array([ 0.72803009])

In [134]:
f2()

array([ 0.55056769])

In [141]:
def copy_random_state(g1, g2):
    if isinstance(g1.rng, MRG_RandomStreams):
        g2.rng.rstate = g1.rng.rstate

    for (su1, su2) in zip(g1.rng.state_updates, g2.rng.state_updates):
            su2[0].set_value(su1[0].get_value())


In [142]:
copy_random_state(g1, g2)

In [143]:
f1()

array([ 0.59044123])

In [144]:
f2()

array([ 0.59044123])

### A Real Example: Logistic Regression

In [145]:
import numpy
import theano
import theano.tensor as T
rng = numpy.random

N = 400
feats = 784
D = (rng.randn(N, feats), rng.randint(size=N, low=0, high=2))
training_steps = 10000

# Declare Theano symbolic variables
x = T.matrix("x")
y = T.vector("y")
w = theano.shared(rng.randn(feats), name="w")
b = theano.shared(0., name="b")
print("Initial model:")
print(w.get_value())
print(b.get_value())

# Construct Theano expression graph
p_1 = 1 / (1 + T.exp(-T.dot(x, w) - b))   # Probability that target = 1
prediction = p_1 > 0.5                    # The prediction thresholded
xent = -y * T.log(p_1) - (1-y) * T.log(1-p_1) # Cross-entropy loss function
cost = xent.mean() + 0.01 * (w ** 2).sum()# The cost to minimize
gw, gb = T.grad(cost, [w, b])             # Compute the gradient of the cost
                                          # (we shall return to this in a
                                          # following section of this tutorial)

# Compile
train = theano.function(
          inputs=[x,y],
          outputs=[prediction, xent],
          updates=((w, w - 0.1 * gw), (b, b - 0.1 * gb)))
predict = theano.function(inputs=[x], outputs=prediction)

# Train
for i in range(training_steps):
    pred, err = train(D[0], D[1])

print("Final model:")
print(w.get_value())
print(b.get_value())
print("target values for D:")
print(D[1])
print("prediction on D:")
print(predict(D[0]))

Initial model:
[  9.06758152e-02   4.00928185e-02   1.11808216e+00   9.13744451e-01
   1.30221732e+00   3.98241423e-01   6.23540092e-02   5.44077516e-01
   7.40056428e-01  -1.51256692e+00   2.25967114e-01  -9.19647131e-01
   7.63932593e-01   1.36089408e+00   6.79009011e-01   6.24442332e-01
  -9.55428145e-01  -1.27901343e+00   3.12743505e-01   7.92546353e-01
  -2.13798343e+00   4.20401440e-01   4.20413807e-01  -3.53432915e-01
  -7.31978226e-01  -1.41952861e+00  -7.56163604e-01   1.72774674e+00
  -6.81686224e-01  -1.37132012e-01   5.92700887e-02   4.42252355e-01
   1.29242933e+00   2.00481204e+00   8.94708776e-01  -9.14198144e-01
  -6.59620972e-01   1.61678967e-01  -1.28389501e+00   1.74341775e-01
  -7.74523325e-01   2.79644137e-01  -6.35077275e-01  -2.18323918e-01
   3.55620215e-01  -4.99954075e-01   5.72552617e-01  -7.36392808e-01
  -1.94283785e+00  -9.52405403e-01   5.39731695e-01  -1.51761240e+00
   9.46303080e-01   1.61681936e+00  -6.10328692e-01  -4.23646006e-02
   1.00800438e+00  